In [2]:
### header ###
__author__ = "Hunter Bennett"
__license__ = "BSD"
__email__ = "hunter.r.bennett@gmail.com"
%load_ext autoreload
%autoreload 2
### imports ###
import sys
%matplotlib inline
import os
import re
import glob
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 200
sns.set(font_scale=1)
sns.set_context('talk')
sns.set_style('white')

# import custom functions
import sys
sys.path.insert(0, '/home/h1bennet/code/')
from hbUtils import ngs_qc, quantile_normalize_df
from plotting_scripts import label_point, pca_rpkm_mat
from homer_preprocessing import read_annotated_peaks

Inge's annotate peak files located:

    /home/iholtman/nuclei_project/annotation_files/annotation_18_7_2019

The file that might work:

    annotated_peaks_tss_with_pooled_tagdirs_H3K27ac_norm.txt

Set working directory

In [3]:
dataDirectory = '/data/mm10/Brain_MPSIIIA/ChIP/H3K27AC/'
inputDirectory = '/data/mm10/Brain_MPSIIIA/ChIP/input/'
workingDirectory = '/home/h1bennet/brain_aging/results/04_Young_Cell_H3K27Ac/'
if not os.path.isdir(workingDirectory):
    os.mkdir(workingDirectory)
os.chdir(workingDirectory)

read in color scheme dictionary

In [4]:
hex_dict={}
rgb_dict={}
for line in open('/gpfs/data01/glasslab/home/h1bennet/brain_aging/data/brain_aging_color_table.txt','r'):
    split = line.strip().replace('\ufeff','').split('\t')
    # split2 = split[1]
    hex_dict[split[0]] = split[1]
    rgb_dict[split[0]] = split[2]    

In [5]:
tagdirs = ['/data/hg19/Nuclei/ChIP/H3K27ac/human_P63_LHX2nuclei_ChIP_H3K27ac_epilepsy_AN_l20181025_GATCAG_trimmed_combined',
           '/data/hg19/Nuclei/ChIP/H3K27ac/human_P70_LHX2nuclei_ChIP_H3K27ac_epilepsy_AN_l20180912_GTGGCC_trimmed_combined',
           '/data/hg19/Nuclei/ChIP/H3K27ac/human_P78_LHX2nuclei_ChIP_H3K27ac_epilepsy_AN_l20190111_TACAGC_trimmed_combined',
           '/data/hg19/Nuclei/ChIP/H3K27ac/human_P63_NeuNnuclei_ChIP_H3K27ac_epilepsy_AN_l20180724_CGATGT_trimmed_combined',
           '/data/hg19/Nuclei/ChIP/H3K27ac/human_P70_NeuNnuclei_ChIP_H3K27ac_epilepsy_AN_l20180724_GCCAAT_trimmed_combined',
           '/data/hg19/Nuclei/ChIP/H3K27ac/human_P73_NeuNnuclei_ChIP_H3K27ac_epilepsy_AN_l20181025_CGATGT_trimmed_combined',
           '/data/hg19/Nuclei/ChIP/H3K27ac/human_P72_NeuNnuclei_ChIP_H3K27ac_epilepsy_AN_l20180912_GTGAAA_trimmed_combined',
           '/data/hg19/Nuclei/ChIP/H3K27ac/human_P63_Olig2nuclei_ChIP_H3K27ac_epilepsy_AN_l20180517_AGTTCC_trimmed_combined',
           '/data/hg19/Nuclei/ChIP/H3K27ac/human_P70_Olig2nuclei_ChIP_H3K27ac_epilepsy_AN_l20180724_ACAGTG_trimmed_combined',
           '/data/hg19/Nuclei/ChIP/H3K27ac/human_P72_Olig2nuclei_ChIP_H3K27ac_epilepsy_AN_l20180912_GTCCGC_trimmed_combined',
           '/data/hg19/Nuclei/ChIP/H3K27ac/human_P73_Olig2nuclei_ChIP_H3K27ac_epilepsy_AN_l20181025_TGACCA_trimmed_combined',
           '/data/hg19/Nuclei/ChIP/H3K27ac/human_P63_PU1nuclei_ChIP_H3K27ac_epilepsy_AN_l20180517_TAGCTT_trimmed_combined',
           '/data/hg19/Nuclei/ChIP/H3K27ac/human_P70_PU1nuclei_ChIP_H3K27ac_epilepsy_AN_l20180724_TGACCA_trimmed_combined',
           '/data/hg19/Nuclei/ChIP/H3K27ac/human_P72_PU1nuclei_ChIP_H3K27ac_epilepsy_AN_l20180912_GTAGAG_trimmed_combined',
           '/data/hg19/Nuclei/ChIP/H3K27ac/human_P73_PU1nuclei_ChIP_H3K27ac_epilepsy_AN_l20181025_ACAGTG_trimmed_combined']

    mergePeaks ./peak_files/*Olig2*variablewidth* > merged_peaks/Olig2_4month_vw_peaks_merged.txt
    mergePeaks ./peak_files/*NeuN*variablewidth* > merged_peaks/NeuN_4month_vw_peaks_merged.txt
    mergePeaks ./peak_files/*PU1*variablewidth* > merged_peaks/PU1_4month_vw_peaks_merged.txt
    mergePeaks ./peak_files/*SOX9*variablewidth* > merged_peaks/SOX9_4month_vw_peaks_merged.txt
    mergePeaks ./peak_files/*variablewidth* > merged_peaks/vw_peaks_merged.txt
    

### Next: Annotate peaks

In [8]:
%%bash
# delete the existing script file
rm ./annotateHumanPromoters_homer.sh
# create a script file
touch ./annotateHumanPromoters_homer.sh

In [9]:
with open('./annotateHumanPromoters_homer.sh', 'w') as f:
    
    annotate_prom_raw = ['annotatePeaks.pl', 'tss',
                         'hg19', '-norm 1e7', '-d \\\n',
                         ' \\\n'.join(tagdirs), '>',
                         './merged_peaks/human_ann_norm_promoters.txt &\n\n']
    
    annotate_prom_norm = ['annotatePeaks.pl', 'tss',
                         'hg19', '-raw', '-d \\\n',
                         ' \\\n'.join(tagdirs), '>',
                         './merged_peaks/human_ann_raw_promoters.txt &\n\n']

    f.write(' '.join(annotate_prom_raw))    
    f.write(' '.join(annotate_prom_norm))
    
    f.close()